In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
import numpy as np

In [3]:
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [4]:
import tensorflow as tf

In [5]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet

from sklearn.kernel_ridge import KernelRidge

from sklearn.linear_model import HuberRegressor
from sklearn.linear_model import RANSACRegressor

from sklearn.linear_model import MultiTaskElasticNet
from sklearn.linear_model import MultiTaskLasso

In [32]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import HistGradientBoostingRegressor

In [8]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

In [9]:
from tqdm import tqdm
import gc

In [10]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")
sample_submission = pd.read_csv("data/sample_submission.csv")

In [23]:
simple_logs = pd.DataFrame()

In [11]:
def mcrmse(y_true, y_pred):
    colwise_mse = tf.reduce_mean(tf.square(y_true - y_pred), axis=0)
    return tf.reduce_mean(tf.sqrt(colwise_mse), axis=0)

In [12]:
train.head(5)

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5


In [13]:
test

,text_id,full_text
0,0000C359D63E,when a person has no experience on a job their...
1,000BAD50D026,Do you think students would benefit from being...
2,00367BB2546B,"Thomas Jefferson once states that ""it is wonde..."


In [14]:
FEATURE_COLUMNS = ['full_text']
TARGET_COLUMNS = ['cohesion', 'syntax','vocabulary', 'phraseology', 'grammar','conventions']

In [15]:
X, Y = train[FEATURE_COLUMNS], train[TARGET_COLUMNS]


In [16]:
kfold = KFold(n_splits=5)

#### LinReg
Mean score: 0.550

In [25]:
predictions = []
scores = []
for idx, (train_index, test_index) in enumerate(kfold.split(X)):
    X_train, X_test, Y_train, Y_test = X.iloc[train_index], X.iloc[test_index], Y.iloc[train_index], Y.iloc[test_index]
    vectorizer = TfidfVectorizer(ngram_range = (1,2))
    X_train = vectorizer.fit_transform(X_train.full_text.tolist())
    X_test = vectorizer.transform(X_test.full_text.tolist())
    
    model = MultiOutputRegressor(LinearRegression(), n_jobs = -1)
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    score_ = mcrmse(Y_test, y_pred)
    scores.append(score_)
    print(f"Fold #{idx}: {score_}")
    
    test_vectorized = vectorizer.transform(test.full_text.tolist())
    test_prediction = model.predict(test_vectorized)
    predictions.append(test_prediction)
    
print (80*'*')
print(f"\n\tMean score: {np.mean(scores):.4f}")
simple_logs[]
    

Fold #0: 0.5541318246499967
Fold #1: 0.5537700014592977
Fold #2: 0.5470261004584632
Fold #3: 0.544407325985179
Fold #4: 0.5520343641116121
********************************************************************************

	Mean loss: 0.550


In [26]:
# with rounding 
predictions = []
scores = []
for idx, (train_index, test_index) in enumerate(kfold.split(X)):
    X_train, X_test, Y_train, Y_test = X.iloc[train_index], X.iloc[test_index], Y.iloc[train_index], Y.iloc[test_index]
    vectorizer = TfidfVectorizer(ngram_range = (1,2))
    X_train = vectorizer.fit_transform(X_train.full_text.tolist())
    X_test = vectorizer.transform(X_test.full_text.tolist())
    
    model = MultiOutputRegressor(LinearRegression(), n_jobs = -1)
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    y_pred = np.round(y_pred*2) / 2
    score_ = mcrmse(Y_test, y_pred)
    scores.append(score_)
    print(f"Fold #{idx}: {score_}")
    
    test_vectorized = vectorizer.transform(test.full_text.tolist())
    test_prediction = model.predict(test_vectorized)
    predictions.append(test_prediction)
    
print (80*'*')
print(f"\n\tMean score: {np.mean(scores):.4f}")
    

Fold #0: 0.5709403512789716
Fold #1: 0.5726226508335558
Fold #2: 0.5635645363980998
Fold #3: 0.5632515029920215
Fold #4: 0.5698789714147965
********************************************************************************

	Mean loss: 0.568


#### ElasticNet
Mean score: .653

In [27]:
predictions = []
scores = []
for idx, (train_index, test_index) in enumerate(kfold.split(X)):
    X_train, X_test, Y_train, Y_test = X.iloc[train_index], X.iloc[test_index], Y.iloc[train_index], Y.iloc[test_index]
    vectorizer = TfidfVectorizer(ngram_range = (1,2))
    X_train = vectorizer.fit_transform(X_train.full_text.tolist())
    X_test = vectorizer.transform(X_test.full_text.tolist())
    
    # X_train, X_test = X_train.toarray(), X_test.toarray()

    model = MultiOutputRegressor(ElasticNet(), n_jobs = -1)
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    score_ = mcrmse(Y_test, y_pred)
    scores.append(score_)
    print(f"Fold #{idx}: {score_}")
    
    test_vectorized = vectorizer.transform(test.full_text.tolist())
    # test_vectorized = test_vectorized.toarray()
    test_prediction = model.predict(test_vectorized)
    predictions.append(test_prediction)
    
print (80*'*')
print(f"\n\tMean score: {np.mean(scores):.4f}")

Fold #0: 0.6528782028635198
Fold #1: 0.6741904406848621
Fold #2: 0.6446424251073121
Fold #3: 0.6458552901536275
Fold #4: 0.6467219977663213
********************************************************************************

	Mean loss: 0.653


#### Kernel ridge 
Mean score: 0.683

In [28]:
predictions = []
scores = []
for idx, (train_index, test_index) in enumerate(kfold.split(X)):
    X_train, X_test, Y_train, Y_test = X.iloc[train_index], X.iloc[test_index], Y.iloc[train_index], Y.iloc[test_index]
    vectorizer = TfidfVectorizer(ngram_range = (1,2))
    X_train = vectorizer.fit_transform(X_train.full_text.tolist())
    X_test = vectorizer.transform(X_test.full_text.tolist())
    
    # X_train, X_test = X_train.toarray(), X_test.toarray()

    model = MultiOutputRegressor(KernelRidge(), n_jobs = -1)
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    score_ = mcrmse(Y_test, y_pred)
    scores.append(score_)
    print(f"Fold #{idx}: {score_}")
    
    test_vectorized = vectorizer.transform(test.full_text.tolist())
    # test_vectorized = test_vectorized.toarray()
    test_prediction = model.predict(test_vectorized)
    predictions.append(test_prediction)
    
print (80*'*')
print(f"\n\tMean score: {np.mean(scores):.4f}")

Fold #0: 0.6930777265189026
Fold #1: 0.6750998745559542
Fold #2: 0.6843776292717582
Fold #3: 0.6704404514406113
Fold #4: 0.6934175409997669
********************************************************************************

	Mean loss: 0.683


#### Huber
Mean score: 0.5478 <br>
tfidf (1,2) best

In [40]:
predictions = []
scores = []
for idx, (train_index, test_index) in enumerate(kfold.split(X)):
    X_train, X_test, Y_train, Y_test = X.iloc[train_index], X.iloc[test_index], Y.iloc[train_index], Y.iloc[test_index]
    vectorizer = TfidfVectorizer(ngram_range = (1,2))
    X_train = vectorizer.fit_transform(X_train.full_text.tolist())
    X_test = vectorizer.transform(X_test.full_text.tolist())
    
    model = MultiOutputRegressor(HuberRegressor(max_iter=100))
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    score_ = mcrmse(Y_test, y_pred)
    scores.append(score_)
    print(f"Fold #{idx}: {score_}")
    
    test_vectorized = vectorizer.transform(test.full_text.tolist())
    test_prediction = model.predict(test_vectorized)
    predictions.append(test_prediction)
    
print (80*'*')
print(f"\n\tMean score: {np.mean(scores):.4f}")

/home/hdd_n/mambaforge/envs/exp/lib/python3.9/site-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Fold #0: 0.5515620102984613


/home/hdd_n/mambaforge/envs/exp/lib/python3.9/site-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Fold #1: 0.5504950203190537


/home/hdd_n/mambaforge/envs/exp/lib/python3.9/site-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Fold #2: 0.5446898913826562


/home/hdd_n/mambaforge/envs/exp/lib/python3.9/site-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Fold #3: 0.5419538049081577


/home/hdd_n/mambaforge/envs/exp/lib/python3.9/site-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Fold #4: 0.5502569124296768
********************************************************************************

	Mean score: 0.5478


In [41]:
predictions = []
scores = []
for idx, (train_index, test_index) in enumerate(kfold.split(X)):
    X_train, X_test, Y_train, Y_test = X.iloc[train_index], X.iloc[test_index], Y.iloc[train_index], Y.iloc[test_index]
    vectorizer = TfidfVectorizer(ngram_range = (1,1))
    X_train = vectorizer.fit_transform(X_train.full_text.tolist())
    X_test = vectorizer.transform(X_test.full_text.tolist())
    
    model = MultiOutputRegressor(HuberRegressor(max_iter=100))
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    score_ = mcrmse(Y_test, y_pred)
    scores.append(score_)
    print(f"Fold #{idx}: {score_}")
    
    test_vectorized = vectorizer.transform(test.full_text.tolist())
    test_prediction = model.predict(test_vectorized)
    predictions.append(test_prediction)
    
print (80*'*')
print(f"\n\tMean score: {np.mean(scores):.4f}")

/home/hdd_n/mambaforge/envs/exp/lib/python3.9/site-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Fold #0: 0.6331420093762457


/home/hdd_n/mambaforge/envs/exp/lib/python3.9/site-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Fold #1: 0.630056943661134


/home/hdd_n/mambaforge/envs/exp/lib/python3.9/site-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Fold #2: 0.626796640564493


/home/hdd_n/mambaforge/envs/exp/lib/python3.9/site-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Fold #3: 0.6249290599158402


/home/hdd_n/mambaforge/envs/exp/lib/python3.9/site-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Fold #4: 0.6326062592100771
********************************************************************************

	Mean score: 0.6295


In [42]:
predictions = []
scores = []
for idx, (train_index, test_index) in enumerate(kfold.split(X)):
    X_train, X_test, Y_train, Y_test = X.iloc[train_index], X.iloc[test_index], Y.iloc[train_index], Y.iloc[test_index]
    vectorizer = TfidfVectorizer(ngram_range = (1,3))
    X_train = vectorizer.fit_transform(X_train.full_text.tolist())
    X_test = vectorizer.transform(X_test.full_text.tolist())
    
    model = MultiOutputRegressor(HuberRegressor(max_iter=100))
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    score_ = mcrmse(Y_test, y_pred)
    scores.append(score_)
    print(f"Fold #{idx}: {score_}")
    
    test_vectorized = vectorizer.transform(test.full_text.tolist())
    test_prediction = model.predict(test_vectorized)
    predictions.append(test_prediction)
    
print (80*'*')
print(f"\n\tMean score: {np.mean(scores):.4f}")

/home/hdd_n/mambaforge/envs/exp/lib/python3.9/site-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Fold #0: 0.5538161068513142


/home/hdd_n/mambaforge/envs/exp/lib/python3.9/site-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Fold #1: 0.5509624993104995


/home/hdd_n/mambaforge/envs/exp/lib/python3.9/site-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Fold #2: 0.5509353575657568


/home/hdd_n/mambaforge/envs/exp/lib/python3.9/site-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Fold #3: 0.5418378324264046


/home/hdd_n/mambaforge/envs/exp/lib/python3.9/site-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Fold #4: 0.5560860014742328
********************************************************************************

	Mean score: 0.5507


#### MultiTaskLasso
Mean score: 0.653

In [19]:
predictions = []
scores = []
for idx, (train_index, test_index) in enumerate(kfold.split(X)):
    X_train, X_test, Y_train, Y_test = X.iloc[train_index], X.iloc[test_index], Y.iloc[train_index], Y.iloc[test_index]
    vectorizer = TfidfVectorizer(ngram_range = (1,2))
    X_train = vectorizer.fit_transform(X_train.full_text.tolist())
    X_test = vectorizer.transform(X_test.full_text.tolist())
    X_train, X_test = X_train.toarray(), X_test.toarray()
    
    model = MultiTaskLasso()
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    score_ = mcrmse(Y_test, y_pred)
    scores.append(score_)
    print(f"Fold #{idx}: {score_}")
    
    test_vectorized = vectorizer.transform(test.full_text.tolist())
    test_prediction = model.predict(test_vectorized)
    test_vectorized = test_vectorized.toarray()

    predictions.append(test_prediction)
    
print (80*'*')
print(f"\n\tMean score: {np.mean(scores):.4f}")

Fold #0: 0.6528782028635198
Fold #1: 0.6741904406848621
Fold #2: 0.6446424251073121
Fold #3: 0.6458552901536275
Fold #4: 0.6467219977663213
********************************************************************************

	Mean loss: 0.653


#### Decision Tree
Mean score: 0.807

In [16]:
predictions = []
scores = []
for idx, (train_index, test_index) in enumerate(kfold.split(X)):
    X_train, X_test, Y_train, Y_test = X.iloc[train_index], X.iloc[test_index], Y.iloc[train_index], Y.iloc[test_index]
    vectorizer = TfidfVectorizer(ngram_range = (1,2))
    X_train = vectorizer.fit_transform(X_train.full_text.tolist())
    X_test = vectorizer.transform(X_test.full_text.tolist())
    
    model = DecisionTreeRegressor()
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    score_ = mcrmse(Y_test, y_pred)
    scores.append(score_)
    print(f"Fold #{idx}: {score_}")
    
    test_vectorized = vectorizer.transform(test.full_text.tolist())
    test_prediction = model.predict(test_vectorized)
    predictions.append(test_prediction)
    
print (80*'*')
print(f"\n\tMean score: {np.mean(scores):.4f}")

2022-09-15 04:36:22.455220: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-15 04:36:22.471719: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-09-15 04:36:22.471730: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-09-15 04:36:22.472148: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN

Fold #0: 0.7931361523412584
Fold #1: 0.8144465691907875
Fold #2: 0.8023388564277685
Fold #3: 0.8114694681488267
Fold #4: 0.8132698138502844
********************************************************************************

	Mean loss: 0.807


#### Random Forest
Mean score: 0.569

In [ ]:
predictions = []
scores = []
for idx, (train_index, test_index) in enumerate(kfold.split(X)):
    X_train, X_test, Y_train, Y_test = X.iloc[train_index], X.iloc[test_index], Y.iloc[train_index], Y.iloc[test_index]
    vectorizer = TfidfVectorizer(ngram_range = (1,2))
    X_train = vectorizer.fit_transform(X_train.full_text.tolist())
    X_test = vectorizer.transform(X_test.full_text.tolist())
    
    model = RandomForestRegressor(n_jobs = -1)
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    score_ = mcrmse(Y_test, y_pred)
    scores.append(score_)
    print(f"Fold #{idx}: {score_}")
    
    test_vectorized = vectorizer.transform(test.full_text.tolist())
    test_prediction = model.predict(test_vectorized)
    predictions.append(test_prediction)
    
print (80*'*')
print(f"\n\tMean score: {np.mean(scores):.4f}")

#### XGB
Mean score: 0.582

In [17]:
predictions = []
scores = []
for idx, (train_index, test_index) in enumerate(kfold.split(X)):
    X_train, X_test, Y_train, Y_test = X.iloc[train_index], X.iloc[test_index], Y.iloc[train_index], Y.iloc[test_index]
    vectorizer = TfidfVectorizer(ngram_range = (1,2))
    X_train = vectorizer.fit_transform(X_train.full_text.tolist())
    X_test = vectorizer.transform(X_test.full_text.tolist())
    
    model = XGBRegressor(n_jobs = -1)
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    score_ = mcrmse(Y_test, y_pred)
    scores.append(score_)
    print(f"Fold #{idx}: {score_}")
    
    test_vectorized = vectorizer.transform(test.full_text.tolist())
    test_prediction = model.predict(test_vectorized)
    predictions.append(test_prediction)
    
print (80*'*')
print(f"\n\tMean score: {np.mean(scores):.4f}")

2022-09-15 04:58:14.703313: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-15 04:58:14.721896: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-09-15 04:58:14.721910: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-09-15 04:58:14.722549: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN

Fold #0: 0.5816267489732073
Fold #1: 0.5901526685793419
Fold #2: 0.5822439205413895
Fold #3: 0.5774044165601089
Fold #4: 0.5763746517735759
********************************************************************************

	Mean loss: 0.582


#### LGBM
Mean score: 0.550

In [21]:
predictions = []
scores = []
for idx, (train_index, test_index) in enumerate(kfold.split(X)):
    X_train, X_test, Y_train, Y_test = X.iloc[train_index], X.iloc[test_index], Y.iloc[train_index], Y.iloc[test_index]
    vectorizer = TfidfVectorizer(ngram_range = (1,2))
    X_train = vectorizer.fit_transform(X_train.full_text.tolist())
    X_test = vectorizer.transform(X_test.full_text.tolist())
    
    model = MultiOutputRegressor(LGBMRegressor(n_jobs = -1))
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    score_ = mcrmse(Y_test, y_pred)
    scores.append(score_)
    print(f"Fold #{idx}: {score_}")
    
    test_vectorized = vectorizer.transform(test.full_text.tolist())
    test_prediction = model.predict(test_vectorized)
    predictions.append(test_prediction)
    
print (80*'*')
print(f"\n\tMean score: {np.mean(scores):.4f}")

Fold #0: 0.5517973919318183
Fold #1: 0.5551591652486675
Fold #2: 0.5483291446127709
Fold #3: 0.5482365804964576
Fold #4: 0.5465152434993047
********************************************************************************

	Mean score: 0.550


In [22]:
predictions = []
scores = []
for idx, (train_index, test_index) in enumerate(kfold.split(X)):
    X_train, X_test, Y_train, Y_test = X.iloc[train_index], X.iloc[test_index], Y.iloc[train_index], Y.iloc[test_index]
    vectorizer = TfidfVectorizer(ngram_range = (1,1))
    X_train = vectorizer.fit_transform(X_train.full_text.tolist())
    X_test = vectorizer.transform(X_test.full_text.tolist())
    
    model = MultiOutputRegressor(LGBMRegressor(n_jobs = -1))
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    score_ = mcrmse(Y_test, y_pred)
    scores.append(score_)
    print(f"Fold #{idx}: {score_}")
    
    test_vectorized = vectorizer.transform(test.full_text.tolist())
    test_prediction = model.predict(test_vectorized)
    predictions.append(test_prediction)
    
print (80*'*')
print(f"\n\tMean score: {np.mean(scores):.4f}")

Fold #0: 0.5569290386975966
Fold #1: 0.5516524187961335
Fold #2: 0.5498964155109362
Fold #3: 0.5458818916232535
Fold #4: 0.5478695454500438
********************************************************************************

	Mean score: 0.550


#### KNN
Mean score: 0.6645

In [31]:
predictions = []
scores = []
for idx, (train_index, test_index) in enumerate(kfold.split(X)):
    X_train, X_test, Y_train, Y_test = X.iloc[train_index], X.iloc[test_index], Y.iloc[train_index], Y.iloc[test_index]
    vectorizer = TfidfVectorizer(ngram_range = (1,2))
    X_train = vectorizer.fit_transform(X_train.full_text.tolist())
    X_test = vectorizer.transform(X_test.full_text.tolist())
    
    model = KNeighborsRegressor(n_jobs = -1)
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    score_ = mcrmse(Y_test, y_pred)
    scores.append(score_)
    print(f"Fold #{idx}: {score_}")
    
    test_vectorized = vectorizer.transform(test.full_text.tolist())
    test_prediction = model.predict(test_vectorized)
    predictions.append(test_prediction)
    
print (80*'*')
print(f"\n\tMean score: {np.mean(scores):.4f}")

Fold #0: 0.671892799169405
Fold #1: 0.67549296640102
Fold #2: 0.6435011356635894
Fold #3: 0.6580667474425836
Fold #4: 0.6736422701792272
********************************************************************************

	Mean score: 0.6645


#### SVR
Mean score: 0.5507

In [29]:
predictions = []
scores = []
for idx, (train_index, test_index) in enumerate(kfold.split(X)):
    X_train, X_test, Y_train, Y_test = X.iloc[train_index], X.iloc[test_index], Y.iloc[train_index], Y.iloc[test_index]
    vectorizer = TfidfVectorizer(ngram_range = (1,2))
    X_train = vectorizer.fit_transform(X_train.full_text.tolist())
    X_test = vectorizer.transform(X_test.full_text.tolist())
    
    model = MultiOutputRegressor(SVR())
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    score_ = mcrmse(Y_test, y_pred)
    scores.append(score_)
    print(f"Fold #{idx}: {score_}")
    
    test_vectorized = vectorizer.transform(test.full_text.tolist())
    test_prediction = model.predict(test_vectorized)
    predictions.append(test_prediction)
    
print (80*'*')
print(f"\n\tMean score: {np.mean(scores):.4f}")

Fold #0: 0.5527417883782569
Fold #1: 0.5598486346576218
Fold #2: 0.5466700014211566
Fold #3: 0.5447914627070536
Fold #4: 0.549511123877991
********************************************************************************

	Mean score: 0.5507


In [30]:
predictions = []
scores = []
for idx, (train_index, test_index) in enumerate(kfold.split(X)):
    X_train, X_test, Y_train, Y_test = X.iloc[train_index], X.iloc[test_index], Y.iloc[train_index], Y.iloc[test_index]
    vectorizer = TfidfVectorizer(ngram_range = (1,1))
    X_train = vectorizer.fit_transform(X_train.full_text.tolist())
    X_test = vectorizer.transform(X_test.full_text.tolist())
    
    model = MultiOutputRegressor(SVR())
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    score_ = mcrmse(Y_test, y_pred)
    scores.append(score_)
    print(f"Fold #{idx}: {score_}")
    
    test_vectorized = vectorizer.transform(test.full_text.tolist())
    test_prediction = model.predict(test_vectorized)
    predictions.append(test_prediction)
    
print (80*'*')
print(f"\n\tMean score: {np.mean(scores):.4f}")

Fold #0: 0.5534128628705631
Fold #1: 0.5568149787750852
Fold #2: 0.5462140456365606
Fold #3: 0.5468043613997916
Fold #4: 0.5493505076755353
********************************************************************************

	Mean score: 0.5505


#### HistGradient

In [37]:
predictions = []
scores = []
for idx, (train_index, test_index) in enumerate(kfold.split(X)):
    X_train, X_test, Y_train, Y_test = X.iloc[train_index], X.iloc[test_index], Y.iloc[train_index], Y.iloc[test_index]
    vectorizer = TfidfVectorizer(ngram_range = (1,1))
    X_train = vectorizer.fit_transform(X_train.full_text.tolist())
    X_test = vectorizer.transform(X_test.full_text.tolist())
    X_train, X_test = X_train.toarray(), X_test.toarray()

    model = MultiOutputRegressor(HistGradientBoostingRegressor())
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    score_ = mcrmse(Y_test, y_pred)
    scores.append(score_)
    print(f"Fold #{idx}: {score_}")
    
    test_vectorized = vectorizer.transform(test.full_text.tolist())
    test_vectorized = test_vectorized.toarray()
    test_prediction = model.predict(test_vectorized)
    predictions.append(test_prediction)
    
print (80*'*')
print(f"\n\tMean score: {np.mean(scores):.4f}")

Fold #0: 0.5537410728071898


KeyboardInterrupt: 

#### Submission

In [ ]:
preds_mean = np.mean(predictions, axis = 0)
submission = pd.DataFrame()
submission.loc[:,'text_id'] = test.text_id
submission.loc[:, TARGET_COLUMNS] = preds_mean

# submission.loc[:, TARGET_COLUMNS] = (submission.loc[:, TARGET_COLUMNS] * 2).round() / 2
submission

,text_id,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0000C359D63E,3.066927,2.759472,3.144001,3.051319,2.492089,2.823449
1,000BAD50D026,2.447473,2.554104,2.579586,2.329913,2.287815,2.528216
2,00367BB2546B,3.582943,3.397358,3.529512,3.458739,3.600916,3.366636


In [ ]:
submission.to_csv('submission.csv', index=False)

In [ ]:
# Score .54 LOL